In [ ]:
from google.colab import drive 
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive/project_nlp

### Install package

In [1]:
!git clone https://github.com/huggingface/transformers.git
%cd transformers
!pip install --upgrade . -q

Cloning into 'transformers'...
remote: Enumerating objects: 91582, done.
remote: Counting objects: 100% (38/38), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 91582 (delta 8), reused 17 (delta 1), pack-reused 91544
Receiving objects: 100% (91582/91582), 75.68 MiB | 13.22 MiB/s, done.
Resolving deltas: 100% (66087/66087), done.
/content/transformers
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 61 kB 546 kB/s 
     |████████████████████████████████| 895 kB 7

In [2]:
!gdown https://drive.google.com/a/gm.uit.edu.vn/uc?id=1pXJZ9eHp6DWkQ5MhCzmWYsKyLQEDiodz&export=download
!tar xzf /content/vn_sbert_deploy.tar.gz

Downloading...
From: https://drive.google.com/a/gm.uit.edu.vn/uc?id=1pXJZ9eHp6DWkQ5MhCzmWYsKyLQEDiodz
To: /content/transformers/vn_sbert_deploy.tar.gz
100% 485M/485M [00:03<00:00, 140MB/s] 
tar (child): /content/vn_sbert_deploy.tar.gz: Cannot open: No such file or directory
tar (child): Error is not recoverable: exiting now
tar: Child returned status 2
tar: Error is not recoverable: exiting now


In [3]:
!pip -q install fastapi
!pip -q install uvicorn
!pip -q install pyngrok

!git clone https://github.com/DatCanCode/sentence-transformers
!cd sentence-transformers
!pip install -e . -q
!pip install sentence_transformers -q
!pip install underthesea -q

!pip -q install vncorenlp
!pip -q install fairseq
!pip -q install fastBPE

!pip install underthesea -q
!underthesea download-data SE_Vietnamese-UB

     |████████████████████████████████| 51 kB 729 kB/s 
     |████████████████████████████████| 10.1 MB 7.9 MB/s 
     |████████████████████████████████| 61 kB 336 kB/s 
     |████████████████████████████████| 78 kB 8.8 MB/s 
     |████████████████████████████████| 54 kB 2.2 MB/s 
     |████████████████████████████████| 54 kB 4.0 MB/s 
     |████████████████████████████████| 745 kB 4.2 MB/s 
Cloning into 'sentence-transformers'...
remote: Enumerating objects: 1200, done.
remote: Total 1200 (delta 0), reused 0 (delta 0), pack-reused 1200
Receiving objects: 100% (1200/1200), 30.67 MiB | 11.92 MiB/s, done.
Resolving deltas: 100% (820/820), done.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 78 kB 3.4 MB/s 
     |████████████████████████████████| 1.2 MB 22.9 MB/s 
     |████████████████████████████████| 7.5 MB 3.8 MB/s 
     |████████████████████████████████| 43 kB 2.5 MB

REF: https://github.com/DatCanCode/sentence-transformers

In [4]:
# Install the vncorenlp python wrapper
!pip install vncorenlp -q

# Download VnCoreNLP-1.1.1.jar & its word segmentation component (i.e. RDRSegmenter) 
!mkdir -p vncorenlp/models/wordsegmenter
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/vi-vocab
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/wordsegmenter.rdr
!mv VnCoreNLP-1.1.1.jar vncorenlp/ 
!mv vi-vocab vncorenlp/models/wordsegmenter/
!mv wordsegmenter.rdr vncorenlp/models/wordsegmenter/

--2021-12-13 13:28:24--  https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 27412575 (26M) [application/octet-stream]
Saving to: ‘VnCoreNLP-1.1.1.jar’

VnCoreNLP-1.1.1.jar 100%[===================>]  26.14M   137MB/s    in 0.2s    

2021-12-13 13:28:27 (137 MB/s) - ‘VnCoreNLP-1.1.1.jar’ saved [27412575/27412575]

--2021-12-13 13:28:27--  https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/vi-vocab
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting respon

## Sentence embedding

In [ ]:
import numpy as np
import itertools

def mmr(doc_embedding, word_embeddings, words, top_n, diversity):

    # Extract similarity within words, and between words and the document
    word_doc_similarity = cosine_similarity(word_embeddings, doc_embedding)
    word_similarity = cosine_similarity(word_embeddings)

    # Initialize candidates and already choose best keyword/keyphras
    keywords_idx = [np.argmax(word_doc_similarity)]
    candidates_idx = [i for i in range(len(words)) if i != keywords_idx[0]]

    for _ in range(top_n - 1):
        # Extract similarities within candidates and
        # between candidates and selected keywords/phrases
        candidate_similarities = word_doc_similarity[candidates_idx, :]
        target_similarities = np.max(word_similarity[candidates_idx][:, keywords_idx], axis=1)

        # Calculate MMR
        mmr = (1-diversity) * candidate_similarities - diversity * target_similarities.reshape(-1, 1)
        mmr_idx = candidates_idx[np.argmax(mmr)]

        # Update keywords & candidates
        keywords_idx.append(mmr_idx)
        candidates_idx.remove(mmr_idx)

    return [words[idx] for idx in keywords_idx]

def max_sum_sim(doc_embedding, word_embeddings, top_n, nr_candidates):
    # Calculate distances and extract keywords
    distances = cosine_similarity(doc_embedding, candidate_embeddings)
    distances_candidates = cosine_similarity(candidate_embeddings, 
                                            candidate_embeddings)

    # Get top_n words as candidates based on cosine similarity
    words_idx = list(distances.argsort()[0][-nr_candidates:])
    words_vals = [candidates[index] for index in words_idx]
    distances_candidates = distances_candidates[np.ix_(words_idx, words_idx)]

    # Calculate the combination of words that are the least similar to each other
    min_sim = np.inf
    candidate = None
    for combination in itertools.combinations(range(len(words_idx)), top_n):
        sim = sum([distances_candidates[i][j] for i in combination for j in combination if i != j])
        if sim < min_sim:
            candidate = combination
            min_sim = sim

    return [words_vals[idx] for idx in candidate]

### Candidates from corpus

In [ ]:
import pandas as pd 
df = pd.read_csv('/content/datafull.csv')
df.head()

,content,star
0,Hàng ko đúng chất liệu như mô tả. Hàng báo là ...,1
1,gian lan . toi mua siz xxl khi giao siz s ma b...,1
2,"Chất lượng sản phẩm quá tệ, không giống miêu t...",1
3,tiki cùi bắp 3 đơn hàng đặt 1 tháng ko nhúc nh...,1
4,"Sản phẩm ko giống như shop quảng cáo, chất liệ...",1


In [ ]:
docs = df.content.values.tolist()
docs

['Hàng ko đúng chất liệu như mô tả. Hàng báo là chất liệu cotton nhưng nhận hàng chất liệu toàn Nylon. Sản phẩm bảo của May 10 nhưng mác trên quần ghi theo thiết kế may 10. Có thể trả lại sản phẩm không shop?',
 'gian lan . toi mua siz xxl khi giao siz s ma bao thi ghi siz M .kg vo hang thi dung ban .quen thoi luu manh . tin tuong hang viet nam ma lsm mat long tin qua .de nghi may 10 doi hang cho toi gap. len tivi keu lam an kho khan ma khi nguoi ta mua thi gian lan vay hoi ai dam mua nua chu?',
 'Chất lượng sản phẩm quá tệ, không giống miêu tả. Chất liệu toàn Nylon, không phải hàng May10. Có hơn 100k, không đáng bao nhiêu nên không buồn đổi trả hàng. Tiki nên xem lại nhà cung cấp, bán hàng kém chất lượng',
 'tiki cùi bắp 3 đơn hàng đặt 1 tháng ko nhúc nhích hủy được 2 đơn qua shoppee zalada đăt chưa 1 tuân là có rồi, đơn này lỡ thanh toán app mới ko hủy được, h thì xoá app luôn bye tiki',
 'Sản phẩm ko giống như shop quảng cáo, chất liệu toàn Nylon, muốn trả lại hàng thì làm ntn hả sh

In [ ]:
len(docs)

62849

In [ ]:
from underthesea import word_tokenize

docs_seg = [word_tokenize(doc, format='text') for doc in docs]
docs_seg

['Hàng ko đúng chất_liệu như mô_tả . Hàng báo là chất_liệu cotton nhưng nhận hàng chất_liệu toàn Nylon . Sản_phẩm bảo của May 10 nhưng mác trên quần ghi theo thiết_kế may 10 . Có_thể trả lại sản_phẩm không shop ?',
 'gian lan . toi mua siz xxl khi giao siz s ma bao thi ghi siz M . kg vo hang thi dung ban . quen thoi luu manh . tin tuong hang viet nam ma lsm mat long tin qua . de nghi may 10 doi hang cho toi gap . len tivi keu lam an kho khan ma khi nguoi ta mua thi gian lan vay hoi ai dam mua nua chu ?',
 'Chất_lượng sản_phẩm quá tệ , không giống miêu_tả . Chất_liệu toàn Nylon , không phải hàng May10 . Có hơn 100 k , không đáng bao_nhiêu nên không buồn đổi trả hàng . Tiki nên xem lại nhà cung_cấp , bán hàng kém chất_lượng',
 'tiki cùi_bắp 3 đơn hàng đặt 1 tháng ko nhúc_nhích hủy được 2 đơn qua shoppee zalada đăt chưa 1 tuân là có rồi , đơn này lỡ thanh_toán app mới ko hủy được , h thì xoá app luôn bye tiki',
 'Sản_phẩm ko giống như shop quảng_cáo , chất_liệu toàn Nylon , muốn trả lại h

In [ ]:
len(docs_seg)

62849

In [ ]:
with open('/content/vietnamese-stopwords.txt', 'r') as f:
    stopwords = f.readlines()
black_list = ['không']    
stopwords = [word.replace(' ','_') for word in stopwords if len(set(word.split()).intersection(black_list))==0]
print(len(stopwords))
print(stopwords[:10])

1892
['a_lô\n', 'a_ha\n', 'ai\n', 'ai_ai\n', 'ai_nấy\n', 'ai_đó\n', 'alô\n', 'amen\n', 'anh\n', 'anh_ấy\n']


In [ ]:
with open('/content/vietnamese-stopwords-clean.txt', 'w') as f:
    f.writelines(''.join(stopwords))

In [ ]:
stop_words = [word.replace('\n', '') for word in stopwords]
print(stop_words[:10])

['a_lô', 'a_ha', 'ai', 'ai_ai', 'ai_nấy', 'ai_đó', 'alô', 'amen', 'anh', 'anh_ấy']


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
# corpus = '\n '.join(docs)
n_gram_range = (1, 2)
stop_words = [word.replace('\n', '') for word in stopwords]

# Extract candidate words/phrases
count = CountVectorizer(ngram_range=n_gram_range, stop_words=stop_words).fit(docs_seg)
candidates = count.get_feature_names()

print(candidates)
print(len(candidates))

['00', '00 cực', '00 j4b', '00 mua', '000', '000 14', '000 230', '000 244', '000 85', '000 94', '000 _shop', '000 ad', '000 chất_liệu', '000 chỗ', '000 combo', '000 giao', '000 giá', '000 hài_lòng', '000 hàng', '000 không', '000 mua', '000 mặc_dù', '000 nghĩa', '000 nghệ_thuật', '000 quần', '000 tem', '000 thực_chất', '000 tiki', '000 tiếc', '000 vnd', '000 vnd_gd', '000 xấu', '000 áo', '000 đắt', '000 đồng', '000000000000000000000000000000000000000000000000000000000000000000000', '000000000000000000000000000000000000000000000000000000000000000000000 000000000000000000000000000000000000000000000000000000000000000000000', '0071000973702', '0071000973702 vietcombank', '01', '01 02', '01 03', '01 11', '01 12', '01 _tiki', '01 chẳng', '01 combo', '01 gửi', '01 không', '01 mua', '01 màu', '01 móc', '01 pijama', '01 quần', '01 rộng', '01 sản_phẩm', '01 túi', '01 vết', '01 áo', '01 đc', '02', '02 2020', '02 2021', '02 _shop', '02 combo', '02 compo', '02 công_sở', '02 cỡ', '02 ko', '02 màu', '

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
candidate_embeddings = model.encode(candidates)
doc_embeddings = model.encode(docs_seg)
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
docs_seg

NameError: ignored

In [ ]:
import tqdm
docs_keywords = []
for i, doc in tqdm.tqdm(enumerate(docs_seg)):
    distances = cosine_similarity(doc_embeddings[i].reshape(1, -1), candidate_embeddings)

    keywords = max_sum_sim(doc_embeddings[i].reshape(1, -1), candidate_embeddings, top_n=5, nr_candidates=20)
    docs_keywords.append(keywords)
df['keywords'] = docs_keywords

0it [00:00, ?it/s]

In [ ]:
df

In [ ]:
from collections import Counter

In [ ]:
count = Counter()
for keywords in docs_keywords:
    count.update(keywords)

In [ ]:
count

In [ ]:
count_sorted = {k: v for k, v in sorted(count.items(), key=lambda item: item[1])}

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(30,10))
plt.bar(count_sorted.keys(), count_sorted.values(), color='g')
plt.xticks(rotation=90)
plt.show()

In [ ]:
{k: v for k, v in count_sorted.items() if v>10}

{'cotton': 19,
 'giặt': 15,
 'gói': 16,
 'lót': 16,
 'may_vá': 12,
 'mềm_mại': 18,
 'quần_may': 13,
 'quần_đùi': 37,
 'rẻ_tiền': 22,
 'shop': 15,
 'tặng_shop': 22,
 'tủ': 13,
 'vải': 18,
 'áo': 12,
 'áo_dài': 12,
 'đơn_giản': 11}

### candidates from 1 comment

In [ ]:
doc = 'giao hàng nhanh ,mới đặt lúc tối sáng đã giao rồi ,nhân viên giao hàng dễ thương ,nhiệt tình,hàng giao đúng bới hàng đã đặt ,date tới 2023 ,còn được tặng quà nữa ,nói chung rất hài lòng'

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

n_gram_range = (2, 3)
stop_words = None

# Extract candidate words/phrases
count = CountVectorizer(ngram_range=n_gram_range, stop_words=stop_words).fit([doc])
candidates = count.get_feature_names()

print(candidates)

['giao hàng', 'giao hàng nhanh', 'giao rồi', 'hàng nhanh', 'hàng nhanh mới', 'lúc tối', 'lúc tối sáng', 'mới đặt', 'mới đặt lúc', 'nhanh mới', 'nhanh mới đặt', 'sáng đã', 'sáng đã giao', 'tối sáng', 'tối sáng đã', 'đã giao', 'đã giao rồi', 'đặt lúc', 'đặt lúc tối']


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
# temp = CountVectorizer(ngram_range=(1,1), stop_words=stop_words).fit([doc])
# temp = temp.get_feature_names()
# print(len(temp)*0.2)

In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np


model = SentenceTransformer('/content/vn_sbert_deploy/phobert_base_mean_tokens_NLI_STS')

# doc_embedding = model.encode([doc])
# candidate_embeddings = model.encode(candidates)

# from sklearn.metrics.pairwise import cosine_similarity

# top_n = 10
# distances = cosine_similarity(doc_embedding, candidate_embeddings)
# keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]]



In [ ]:
print(keywords)

['lúc tối', 'mới đặt lúc', 'giao rồi', 'nhanh mới', 'giao hàng', 'sáng đã', 'lúc tối sáng', 'sáng đã giao', 'nhanh mới đặt', 'giao hàng nhanh']


In [ ]:
max_sum_sim(doc_embedding, candidate_embeddings, top_n=3, nr_candidates=6)

['giao hàng', 'lúc tối sáng', 'nhanh mới đặt']

In [ ]:
mmr(doc_embedding, candidate_embeddings, candidates, top_n=5, diversity=0.7)

['giao hàng nhanh', 'hàng nhanh mới', 'tối sáng đã', 'lúc tối sáng', 'đặt lúc']

### Tokenize from huggingface with phobert

In [ ]:
text = ['Hàng ko đúng chất liệu như mô tả. Hàng báo là chất liệu cotton nhưng nhận hàng chất liệu toàn Nylon. Sản phẩm bảo của May 10 nhưng mác trên quần ghi theo thiết kế may 10. Có thể trả lại sản phẩm không shop?',
'Hàng rất xấu \
Vải nilong , máy xấu rất đáng buồn khi lần đầu mua bên shop \
Giao hàng hơn 1 tháng 30 ngày mơi nhân được \
Nêm quêt định trả hàng lại cho shop',
'Sản phẩm đẹp, chất vải mềm, đúng như mô tả.\
Tuy nhiên tôi không đồng ý ở chỗ mác giá sản phẩm là 25k/1 quần, và tôi cũng đã xem trên trang web của công ty may  10 cũng với giá cũng là 125k/5 quần, vậy giảm giá 62% đưa ra là ở đâu? \
tôi nghĩ, Tiki nên xem xét lại giá của các mặt  hàng, nhiều nhà bán đã nâng khống giá lên, sau đó lại giảm để lừa người mua rằng mình đã mua rẻ.']

In [ ]:
from vncorenlp import VnCoreNLP

# To perform word segmentation, POS tagging, NER and then dependency parsing
annotator = VnCoreNLP("/content/vncorenlp/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx2g') 

In [ ]:
# Input 
text = "Ông Nguyễn Khắc Chúc  đang làm việc tại Đại học Quốc gia Hà Nội. Bà Lan, vợ ông Chúc, cũng làm việc tại đây."

# To perform word segmentation only
word_segmented_text = annotator.tokenize(text)
word_segmented_text

[['Ông',
  'Nguyễn_Khắc_Chúc',
  'đang',
  'làm_việc',
  'tại',
  'Đại_học',
  'Quốc_gia',
  'Hà_Nội',
  '.'],
 ['Bà',
  'Lan',
  ',',
  'vợ',
  'ông',
  'Chúc',
  ',',
  'cũng',
  'làm_việc',
  'tại',
  'đây',
  '.']]

In [ ]:
from transformers import AutoModel, AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")

# tokens = tokenizer.batch_encode_plus(text,max_length=1000,padding='max_length', truncation=True)
tokens = tokenizer(text, padding='longest', truncation=False, return_tensors="pt")

# text_seq = torch.tensor(tokens['input_ids'])
# text_mask = torch.tensor(tokens['attention_mask'])

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
text_seq = tokens['input_ids']
text_mask = tokens['attention_mask']

In [ ]:
text_seq

tensor([[    0,  1383,  5516,   287,   567,  1934,    42,  4447, 38638,     5,
          1383,   441,     8,   567,  1934, 27994,    51,   154,   119,   567,
          1934,   668, 18981,  2029, 10557, 13834,  3884,  1218,     7,  7646,
           210,    51,  7398,    34,  2747,   701,    63, 14945,  3900,  1800,
          2186,     5,   503,  4623,   505,    44,  4770,  3884,    17, 12231,
          1487,   114,     2,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1],
        [    0,  1383,    59,  1078, 31385, 32633,  2662,     4,   558,  1078,
            59,   463,  1520,    26

In [ ]:
text_mask.shape

torch.Size([3, 111])

In [ ]:
for ids in tokens["input_ids"]:
    print(tokenizer.decode(ids))

<s> Hàng ko đúng chất liệu như mô tả. Hàng báo là chất liệu cotton nhưng nhận hàng chất liệu toàn Nylon. Sản phẩm bảo của May 10 nhưng mác trên quần ghi theo thiết kế may 10. Có thể trả lại sản phẩm không shop? </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
<s> Hàng rất xấu Vải nilong, máy xấu rất đáng buồn khi lần đầu mua bên shop Giao hàng hơn 1 tháng 30 ngày mơi nhân được Nêm quêt định trả hàng lại cho shop </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>